In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec

import numpy as np
import pandas as pd
import os
import shutil
import cv2

import glob2

from numba import jit

colors = [
'black','grey','brown','slategrey','darkviolet','darkmagenta',
'blue','blue','blue',
'red','red','red',
'green','green','green',
'cyan','cyan','cyan',
'orange','orange','orange',
    
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue''darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']


Autosaving every 180 seconds


In [3]:
import numpy as np
data = np.load('/media/cat/12TB/insync_cm3746/paninski_lab/data/spikingforest/bionet_static/static_8x_A_2A/tmp/output/templates.npy')
print (data.shape)

(211, 101, 60)


In [10]:
data = np.fromfile('/media/cat/12TB/insync_cm3746/paninski_lab/data/spikingforest/bionet_static/static_8x_A_2A/data.bin','int16')
data2d = data.reshape( 60, -1).T
#data2d.tofile


In [13]:
data1d = data2d.reshape(-1)
print (data1d.shape)

(1152000000,)


In [16]:
data1d.tofile('/media/cat/12TB/insync_cm3746/paninski_lab/data/spikingforest/bionet_static/static_8x_A_2A/data_python.bin')

In [14]:
data2d = data1d.reshape(-1,60)


In [15]:
for k in range(60):
    plt.plot(data2d[:10000, k]+k*10,c='black')
plt.show()

In [23]:
# parse KS2 units and 
min_spikes = 600/2
max_spikes = 6000*10
good_units = []

for unit in range(len(ptps)):
    if (ptps[unit].shape[0]>min_spikes) and (ptps[unit].shape[0]<max_spikes):
        good_units.append(unit)
        
print ("# of good units: ", len(good_units), " of total KS2 units: ", len(ptps))




# of good units:  87  of total KS2 units:  611


In [5]:
# match 
templates_gt = np.load('/media/cat/12TB/insync_cm3746/paninski_lab/data/spikingforest/bionet_static/static_8x_A_2A/ground_truth/templates.npy')
print (templates_gt.shape)
templates_yass = np.load('/media/cat/12TB/insync_cm3746/paninski_lab/data/spikingforest/bionet_static/static_8x_A_2A/tmp/output/templates.npy')
print (templates_yass.shape)




(708, 101, 60)
(211, 101, 60)


In [35]:
import scipy
import scipy.spatial


def find_spike_match(s1, s2):
    max_dist = 60
    ctr=0
    for s in s1:
        dist= np.min(np.abs(s-s2))
        if dist<=max_dist:
            ctr+=1
    
    return (ctr)

def find_template_match(data_in, match_templates,
                        spike_train_yass,
                        spike_train_gt):
   
    #if iter_%50==0:
    #    print ("unit: ", iter_)
    
    template=data_in[0]
    unit = data_in[1]
        
    template = template.T.ravel()
#     plt.plot(template)
#     plt.show()
#     return (_,_,_,_)

    best_result = 0
    best_roll=0 
    best_match = None
    for t in range(match_templates.shape[0]):
        #if t%250==0:
        #    print (t)
        data2 = match_templates[t]
        for k in range(-50,50,1):
            data_test = np.roll(data2,k,axis=1).T.ravel()
            result = 1-scipy.spatial.distance.cosine(template, data_test)
            if result>0.8:
                print (" good result: ", t, k, result)
            if result>best_result:
                best_roll = k
                best_result = result
                best_match = t

    # compute # matches
    idx_yass = np.where(spike_train_yass==unit)[0]
    times_yass = spike_train_yass[idx_yass,0]
    idx_match = np.where(spike_train_gt==best_match)[0]
    times_match = spike_train_gt[idx_match,0]
    
    print ("times yass: ", times_yass[:20])
    print ("times match: ", times_match[:20])
    # find matches
    tot_matches = find_spike_match(times_yass, times_match)
                
    # compute purity, 
   
    purity =  tot_matches/ idx_yass.shape[0]

    if False:
        plt.plot(template)
        plt.plot(match_templates[best_match].T.ravel())
    
    return (best_match, best_result, best_roll, purity)


templates_gt = np.load('/media/cat/12TB/insync_cm3746/paninski_lab/data/spikingforest/bionet_static/static_8x_A_2A/ground_truth/templates.npy')
print (templates_gt.shape)
ptps_gt = templates_gt.ptp(1).max(1)
templates_yass = np.load('/media/cat/12TB/insync_cm3746/paninski_lab/data/spikingforest/bionet_static/static_8x_A_2A/tmp/output/templates.npy')
print (templates_yass.shape)
ptps_yass = templates_yass.ptp(1).max(1)

spike_train_yass = np.load('/media/cat/12TB/insync_cm3746/paninski_lab/data/spikingforest/bionet_static/static_8x_A_2A/tmp/output/spike_train.npy')
spike_train_gt = np.load('/media/cat/12TB/insync_cm3746/paninski_lab/data/spikingforest/bionet_static/static_8x_A_2A/ground_truth/spike_train.npy')


if True:
    results = []
    #for unit in units[:10]:
    for unit in range(10):
        (best_match, best_cos, best_roll,purity) = find_template_match(list((templates_yass[unit],unit)), 
                                                               templates_gt,
                                                               spike_train_yass,        
                                                               spike_train_gt)
        
        #
        print ("unit: ",unit, ", ptp: ", ptps_yass[unit], 
               " gt_match: ", best_match, ", ptp: ", ptps_gt[best_match], 
               ", best cos: ", best_cos," , roll: ", best_roll,
              ", purity: ", purity)
        
        results.append([best_match, best_result, best_roll])
        
        #break
        #ks_unit = best_match
if False:
    import parmap
    # KS2 vs. Yass
    results = parmap.map(find_template_match, 
                         templates_yass, 
                         templates_gt,
                         pm_processes=16)
    



print ("DONE")



(708, 101, 60)
(211, 101, 60)
times yass:  [  4900   5354  36076  36265  97348 142228 173915 209563 221654 270109
 305354 319317 348243 357236 374813 384101 397426 421193 421381 428233]
times match:  [106045 106224 200469 200647 276567 276745 296161 296340 493350 493514
 540792 540975 806249 806426 822757 822922 869606 869781 892720 892902]
unit:  0 , ptp:  16.914463  gt_match:  159 , ptp:  1.7885473 , best cos:  0.36639052629470825  , roll:  -36 , purity:  0.0
times yass:  [  9121  14237  19058  25064  40560  47474  55108  60085  64782  71168
  79692  80181  87803  94102  96548  97705 104901 113800 119857 124650]
times match:  [106045 106224 200469 200647 276567 276745 296161 296340 493350 493514
 540792 540975 806249 806426 822757 822922 869606 869781 892720 892902]
unit:  1 , ptp:  9.682399  gt_match:  159 , ptp:  1.7885473 , best cos:  0.32616230845451355  , roll:  2 , purity:  0.005751258087706686
times yass:  [  6369  13821  17834  32270  34204  49862  52862  69876  82928  85014


TypeError: find_template_match() missing 2 required positional arguments: 'spike_train_yass' and 'spike_train_gt'

In [21]:
units=np.arange(5)
units = [good_units[3]]

for unit in units:
    fig = plt.figure()
    ax = plt.subplot(131)
    #plt.plot(templates[unit].T)
    plt.plot(wfs_array[unit].mean(0))

    ax = plt.subplot(132)
    #spikes = spikes[idx]
    spikes = times[unit]
    print ("spikes:", spikes.shape, "ptps: ", ptps[unit].shape)
    plt.scatter(spikes/30000., ptps[unit])

    ax=plt.subplot(133)
    max_chan = wfs_array[unit].mean(0).ptp(0).argmax(0)#templates[unit].ptp(0).argmax(0)
    print (wfs_array[unit].shape)

    cmap = cm.get_cmap('viridis',wfs_array[unit].shape[0])
    clrs = cmap(np.arange(wfs_array[unit].shape[0]))
    print (clrs)
    for k in range(0,wfs_array[unit].shape[0],10):
        plt.plot(wfs_array[unit][k].T,c=cmap(k))


plt.show()



spikes: (8210,) ptps:  (8210,)
(8210, 101)
[[0.267004 0.004874 0.329415 1.      ]
 [0.267004 0.004874 0.329415 1.      ]
 [0.267004 0.004874 0.329415 1.      ]
 ...
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]]
